# Exploratory Data Analysis & Data Storytelling for IDS KDDCUP99 Dataset


**Problem:**

Use cleaned KDD99Cup IDS data from data wrangling stage and apply exploratory data analysis to help us i a real-time network intrusion detection system.

**Approach:**

Ask questions. With hypothesis, test if the hypothesis is true.

Using panads, matplotlib & seaborn to plot dist plot, count plot, scatter plot, geodata, etc. to answer the questions.
Ask follow-up qusetions.

Repeated the above procedure until all the question are asked and answered.

**Top Questions:**

1. We can think of the following questions and apply them to our KDD99Cup IDS dataset:

2. What might be the most important features that help in differentiating a network intrusion from a normal activity to help us build a real-time network intrusion detection system?

3. What percentage of observations in training set are redundant & not useful?

4. What are the different categories of attack type?

5. What are the different service types & traffic features of these services used for attack?

6. Are there strong correlations between traffic features of different services and attack type?

7. Could we use hypothesis testing by performing statistical methods on correlated service and traffic features ? 
   Null Hypothesis: The incoming traffic is not an attack
   Alternate Hypothesis: The incoming traffic is an attack

8. Could we use univariate , bivariate or multivariate statistical method for analysis?

9. Could we use better tools to present our EDA/findings?

**Conclusion:**

To be added after completing EDA.

In [2]:
## Import required libraries
import os
import numpy                   as np
import pandas                  as pd
import matplotlib              as pl
import seaborn                 as sns

## Reading .csv files
df = pd.read_csv("data/kddcup-training.csv")

In [3]:
df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,http,SF,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.
